In [7]:
import pandas as pd
import numpy as np
import os
import csv
import matplotlib.pyplot as plt
import hvplot.pandas
from pathlib import Path
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split

csv_data = os.path.join(".","Border_Crossing_Entry_Data.csv")

border_df = pd.read_csv(csv_data)

border_df['Date'] = pd.to_datetime(border_df['Date'].str.strip(), errors='coerce')

#Drop any NAs
border_df = border_df.dropna(subset=['Date'])

#extract the 'year'
border_df['Year'] = border_df['Date'].dt.year

#remove any zeros
border_df = border_df.dropna(subset = ['Value'])
border_df = border_df[border_df['Value'] > 0]

#group by year and port
annual_data = border_df.groupby(['Year'], as_index=False)['Value'].sum()

#Sort the data
annual_data = annual_data.sort_values(by=['Year'])

#create a column for change from year to year
annual_data['Annual Change'] = annual_data['Value'].pct_change()

# Set the first year's Annual Change to 0
annual_data['Annual Change'].iloc[0] = 0

# Convert 'Annual Change' to percentage 
annual_data['Annual Change'] = annual_data['Annual Change'] * 100

#create column which can be used to predict increase or decrease
annual_data['Increase'] = (annual_data['Annual Change'] > 0).astype(int)

# format the 'Annual Change' to display as percentage 
annual_data['Annual Change'] = annual_data['Annual Change'].map('{:.2f}%'.format)

# Convert the 'Increase' column to integer (it's already 0 or 1, but just to be sure)
annual_data['Increase'] = annual_data['Increase'].astype(int)

# Select features and target
X = annual_data['Year'].values.reshape(-1, 1)
y = annual_data['Increase']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#create, fit, and predict model
model = LogisticRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

# Evaluate the model's performance
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.4f}')
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

Accuracy: 0.3333

Classification Report:
              precision    recall  f1-score   support

           0       0.33      1.00      0.50         2
           1       0.00      0.00      0.00         4

    accuracy                           0.33         6
   macro avg       0.17      0.50      0.25         6
weighted avg       0.11      0.33      0.17         6



C:\Users\mhennessy\AppData\Local\Temp\ipykernel_9108\2382217858.py:38: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  annual_data['Annual Change'].iloc[0] = 0
C:\Users\mhennessy\AppData\Local\Temp\ipykernel_9108\2382217858.py:38: SettingWithC

In [8]:
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.9, random_state=42)

#create, fit, and predict model
model = LogisticRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

# Evaluate the model's performance
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.4f}')
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

Accuracy: 0.5926

Classification Report:
              precision    recall  f1-score   support

           0       0.58      0.54      0.56        13
           1       0.60      0.64      0.62        14

    accuracy                           0.59        27
   macro avg       0.59      0.59      0.59        27
weighted avg       0.59      0.59      0.59        27

